In [2]:
import os
import pandas as pd

# 계산에 사용할 디렉토리 경로
calc_dir = r"C:\Users\Playdata\Desktop\Final\model\calculation"

# 두 파일의 경로 설정 (파일명은 실제 파일명에 맞게 수정)
pred_file = os.path.join(calc_dir, "predicted_reviews550.xlsx")
emotion_file = os.path.join(calc_dir, "emotion_analysis550.xlsx")

# 첫 행이 헤더(영화 제목)인 상태로 엑셀 파일 읽기
df_pred = pd.read_excel(pred_file, header=0)
df_emotion = pd.read_excel(emotion_file, header=0)

results = []

# predicted_reviews 파일의 각 열은 "{영화제목}_pred" 형식이라고 가정합니다.
for col in df_pred.columns:
    if isinstance(col, str) and col.endswith("_pred"):  # 열 이름이 문자열일 때만 처리
    # if col.endswith("_pred"):
        # 영화 제목 추출 (예: "MovieA_pred" → "MovieA")
        movie = col[:-len("_pred")]
        # emotion 파일에서는 해당 영화의 열 이름이 "{영화제목}_emotion_point"여야 합니다.
        emotion_col = movie + "_emotion_point"
        if emotion_col in df_emotion.columns:
            # 엑셀 파일의 첫 행은 헤더이므로, 두 번째 행부터 151번째 행까지의 데이터는
            # DataFrame에서는 인덱스 0부터 149까지에 해당합니다.
            series_pred = df_pred[col].iloc[0:150]
            series_emotion = df_emotion[emotion_col].iloc[0:150]
            
            # 숫자로 변환 (숫자가 아닌 값은 NaN으로 처리) 후, NaN 제거
            series_pred = pd.to_numeric(series_pred, errors='coerce')
            series_emotion = pd.to_numeric(series_emotion, errors='coerce')
            valid = series_pred.notna() & series_emotion.notna()
            series_pred = series_pred[valid]
            series_emotion = series_emotion[valid]
            
            if len(series_pred) == 0:
                continue
            
            # 행별 곱 계산 후, 모든 곱의 평균 계산
            product = series_pred * series_emotion
            avg_value = product.mean()
            
            # 선형 변환: [-1, 1] → [0, 10]
            transformed_score = round(((avg_value + 1) / 2) * 10, 1)
            
            results.append({"Movie": movie, "Score": transformed_score})
        else:
            print(f"'{movie}'에 해당하는 emotion_point 열이 없습니다.")
            
# 결과를 DataFrame으로 정리한 후 엑셀 파일로 저장
result_df = pd.DataFrame(results)
output_path = os.path.join(calc_dir, "mumuscore550.xlsx")
result_df.to_excel(output_path, index=False)

print(f"결과가 저장되었습니다: {output_path}")


결과가 저장되었습니다: C:\Users\Playdata\Desktop\Final\model\calculation\mumuscore550.xlsx
